In [28]:
import pyxdf
import numpy as np
from sys import argv
from pathlib import Path

In [29]:
xdf_data_path = Path("xdf_data")

In [30]:
xdf_filepaths = list(xdf_data_path.rglob("*.xdf"))

In [31]:
xdf_data = {str(x).split("/")[1] : pyxdf.load_xdf(str(x)) for x in xdf_filepaths}

Stream 3: Calculated effective sampling rate 30.1718 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1676 Hz is different from specified rate 90.0000 Hz.
Stream 5: Calculated effective sampling rate 30.0871 Hz is different from specified rate 90.0000 Hz.
Stream 5: Calculated effective sampling rate 30.1664 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1437 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1845 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1260 Hz is different from specified rate 90.0000 Hz.
Stream 2: Calculated effective sampling rate 30.1270 Hz is different from specified rate 90.0000 Hz.
Stream 5: Calculated effective sampling rate 30.1097 Hz is different from specified rate 90.0000 Hz.
Stream 3: Calculated effective sampling rate 30.1637 Hz is different from specified rate 90

In [42]:
def separate_trials_expmakers(exp_list):
    raise Exception ("SHOULD NOT REACH HERE!")
    for i, string in enumerate(exp_list):
        print(string)
        if string == "":
            return exp_list[0:i], exp_list[i:]
    raise Exception("Separate trial exp should not reach here!")

def separate_trials_latmarkers(lat_list):
    for i, string in enumerate(lat_list):
        if string == "task_block_start|BOX_BLOCK":
            separate_trials = lat_list[0:i], lat_list[i:]
            print(separate_trials)
            return separate_trials

In [43]:
def collect_participant_trials(pnum):
    participant_trials = {}
    participant_trials["move_cans"] = {}
    participant_trials["move_blocks"] = {}
    
    data, header = xdf_data[f"sub-{pnum:03}"]
    for stream in data:
        stream_series = stream["time_series"]
        stream_name = stream["info"]["name"][0]
        if stream_name == "LatencyMarkers":
            stream_series_np = np.array(stream_series).reshape(len(stream_series))
            trial_cans, trial_blocks = separate_trials_latmarkers(list(stream_series_np))
            participant_trials["move_cans"]["lat_markers"] = trial_cans
            participant_trials["move_blocks"]["lat_markers"] = trial_blocks
        elif stream_name == "ExpMarkers":
            stream_series_np = np.array(stream_series).reshape(len(stream_series))
            trial_cans, trial_blocks = separate_trials_latmarkers(list(stream_series_np))
            participant_trials["move_cans"]["exp_markers"] = trial_cans
            participant_trials["move_blocks"]["exp_markers"] = trial_blocks
    
    return participant_trials

In [44]:
collect_participant_trials(8)

([np.str_('task_block_start|JEBSEN_TAYLOR'), np.str_('practice_session_start'), np.str_('labrecorder_start'), np.str_('practice_start'), np.str_('countdown_start'), np.str_('countdown_3'), np.str_('countdown_2'), np.str_('countdown_1'), np.str_('practice_jebsen_taylor_start'), np.str_('practice_jebsen_taylor_end'), np.str_('practice_end'), np.str_('labrecorder_stop'), np.str_('practice_session_end'), np.str_('labrecorder_start'), np.str_('trial_start'), np.str_('countdown_start'), np.str_('countdown_3'), np.str_('countdown_2'), np.str_('countdown_1'), np.str_('jebsen_taylor_start'), np.str_('jebsen_taylor_end'), np.str_('task_accept'), np.str_('trial_end'), np.str_('labrecorder_stop'), np.str_('labrecorder_start'), np.str_('trial_start'), np.str_('countdown_start'), np.str_('countdown_3'), np.str_('countdown_2'), np.str_('countdown_1'), np.str_('jebsen_taylor_start'), np.str_('jebsen_taylor_end'), np.str_('task_accept'), np.str_('trial_end'), np.str_('labrecorder_stop'), np.str_('labre

TypeError: cannot unpack non-iterable NoneType object